In [3]:
# Import PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Download Shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-23 20:17:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-03-23 20:17:08 (33.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
# Inspect the dataset
with open('input.txt','r', encoding='utf-8') as f:
    text = f.read()
print("Length of dataset in characters:", len(text))
print(text[:1000])

Length of dataset in characters: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunge

In [5]:
# Get the vocabulary i.e. the list of unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
# print(''.join(chars))
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [6]:
# Create a mapping from characters to integers and vice versa
char_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_char = {i:ch for i,ch in enumerate(chars)}
# Encode maps a string to a list of integers corresponding to the characters of the string
encode = lambda s: [char_to_int[c] for c in s]
# Decode is the inverse of encode
decode = lambda l: ''.join([int_to_char[i] for i in l])
print(encode('Trang is beautiful.'))
print(decode(encode('Trang is beautiful.')))

[32, 56, 39, 52, 45, 1, 47, 57, 1, 40, 43, 39, 59, 58, 47, 44, 59, 50, 8]
Trang is beautiful.


In [7]:
# Encode the data and convert to tensor
data = torch.tensor(encode(text),dtype=torch.long) #dtype defaults to torch.float32, torch.long is torch.int64
print(data.shape, data.dtype, len(data))
print(data[:100])

torch.Size([1115394]) torch.int64 1115394
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
# Split the data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
# Hyperparameters
batch_size = 16 # number of independent sequences to process in parallel
block_size = 32 # maximum context length
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64 # embedding dimension
n_head = 4 # number of heads, note that n_embd = n_head * head_size
n_layer = 4
dropout = 0.0

In [10]:
torch.manual_seed(1337)
# Data Loading
# Generate a random batch of batch_size sequences of length block_size
def get_batch(split):
  data = train_data if split=="train" else  val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  #print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

# xb, yb = get_batch("train")
# print(xb,yb)
# # The training examples look like
# # Iterate along batch dimension
# for b in range(batch_size):
#   # Iterate along time dimenstion
#   for t in range(block_size):
#     context = xb[b, :t+1]
#     target = yb[b,t]
#     print(f"Batch {b}: When the input is {context}, the target is {target}.")

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [21]:
# Implement one head of self-attention. Recall self-attention means that the keys and values are produced from the same source as queries.
# In "cross-attention", the queries get produced from x, but the keys and values come from some other external source (e.g. an encoder module).
class Head(nn.Module):
    """ One self-attention head """
    def __init__(self, head_size):
        super().__init__()
        self.head_size = head_size
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape # C = n_embd, T less than or equal block_size, B = batch_size
        q = self.query(x) # (B, T, head_size)
        k = self.key(x) # (B, T, head_size)
        v = self.value(x) # (B,T, head_size)

        # Attention scores: dot products of queries and keys
        wei = q @ k.transpose(-2,-1) * self.head_size**-0.5 # (B, T, T)

        # For decoder attention block, we need to apply a triangular mask, this means we only allow for information from the first token up to
        # & including the present token to be communicated to the present token/character. We also apply softmax to the attention scores.
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        out = wei @ v # (B, T, head_size)
        return out


In [22]:
# Implement parallel multi-head self-attention.
class MultiHeadAttention(nn.Module):
    """ Multiple self-attention heads in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # h(x) has shape (B, T, head_size), so out has shape (B, T, head_size * num_heads)
        out = self.proj(out) # Note that n_embd = head_size * num_heads
        out = self.dropout(out)
        return out

In [23]:
# Implement a transformer block consisting of multi-head self-attention, feed forward (to be defined) and layer norm.
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer Block """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.attention = MultiHeadAttention(n_head, head_size)
        self.feedforward = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # Implement residual connection
        x = x + self.attention(self.ln1(x)) # x.shape is (B, T, C = n_embd)
        x = x + self.feedforward(self.ln2(x))
        return x

In [24]:
# Implement a simple bigram language model.
class BigramLanguageModel(nn.Module):
    """ Simple bigram language model """
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # positional encoding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size) # language model head, final layer producing logits

    def forward(self, idx, targets=None):
        # idx is (B, T) array of indices, the indices range over the vocab size
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C=n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [25]:
# Training
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.3112, val loss 4.3215
step 100: train loss 2.6502, val loss 2.6653
step 200: train loss 2.4783, val loss 2.4862
step 300: train loss 2.3880, val loss 2.4023
step 400: train loss 2.3353, val loss 2.3480
step 500: train loss 2.2800, val loss 2.2872
step 600: train loss 2.2302, val loss 2.2605
step 700: train loss 2.1750, val loss 2.2066
step 800: train loss 2.1346, val loss 2.1704
step 900: train loss 2.1010, val loss 2.1363
step 1000: train loss 2.0706, val loss 2.1258
step 1100: train loss 2.0453, val loss 2.0933
step 1200: train loss 2.0092, val loss 2.0585
step 1300: train loss 1.9868, val loss 2.0380
step 1400: train loss 1.9714, val loss 2.0521
step 1500: train loss 1.9337, val loss 2.0237
step 1600: train loss 1.9192, val loss 2.0050
step 1700: train loss 1.8939, val loss 2.0063
step 1800: train loss 1.8761, val loss 1.9990
step 1900: train loss 1.8641, val loss 1.9841
step 2000: train loss 1.8384, val loss 1.9642
step 2100: train loss 1.